The following is **Meeting minutes Generator** by using **QWEN2** and **Openai Opensource model whisper for transcription**, check the following colab link to see the outputs

https://colab.research.google.com/drive/1_pqFmQXjOYG9Se4Zov4blIGeoYX6ViTJ?usp=sharing


In [0]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


In [0]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
    return_timestamps=True #important if audio is more than 30sec
)

In [0]:
result = pipe("your-audio.mp3")

In [0]:
transcription = result["text"]
print(transcription)

## MODEL

In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

In [0]:
QWEN2 = "Qwen/Qwen2-7B-Instruct"

In [0]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [0]:
tokenizer = AutoTokenizer.from_pretrained(QWEN2)
#tokenizer.pad_token = tokenizer.oes_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_ptrompt=True).to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(QWEN2 , device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)


In [0]:
response = tokenizer.decode(outputs[0])

## **For Markdown display**

In [0]:
from IPython.display import Markdown, display, update_display

In [0]:
display(Markdown(response))